In [ ]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
import skimage.transform as skt
import hyperspy.api as hys
from sklearn.decomposition import NMF
import tkinter.filedialog as tkf
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
# refer to "github.com/mkolopanis/python/blob/master/radialProfile.py"

def radial_average(image, profile_resolution=1.0, center=None):
   
    y, x = np.indices(image.shape) * profile_resolution
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
        
    r = np.hypot(y - center[0]*profile_resolution, x - center[1]*profile_resolution)
    #plt.imshow(r, cmap="Accent")
    #plt.show()

    # Get sorted radii
    ind = np.argsort(r.flat)
    r_sorted = r.flat[ind]
    i_sorted = image.flat[ind]

    # Get the integer part of the radii (bin size = 1)
    r_int = r_sorted.astype(int)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = np.where(deltar)[0]       # location of changed radius
    nr = rind[1:] - rind[:-1]        # number of radius bin
    #print(nr)
    
    csim = np.cumsum(i_sorted, dtype=float)
    tbin = csim[rind[1:]] - csim[rind[:-1]]

    radial_prof = tbin / nr

    return radial_prof

In [ ]:
def warp_scale(spectrum):
    spectrum = spectrum.clip(min=0.0)
    min_val = np.min(spectrum)
    
    rescaled = spectrum - min_val
    
    if np.max(rescaled) != 0:
        rescaled = rescaled / np.max(rescaled)
    
    return rescaled

In [ ]:
dm_files = tkf.askopenfilenames()
print(len(dm_files))
print(dm_files)

In [ ]:
dm_files = list(dm_files)
dm_files.extend(tkf.askopenfilenames())
print(len(dm_files))
print(dm_files)

In [ ]:
dm_files = list(dm_files)
dm_files.extend(tkf.askopenfilenames())
print(len(dm_files))
print(dm_files)

In [ ]:
# variance map radial average
data_storage = []
for i, adr in enumerate(dm_files):
    temp = hys.load(adr)
    
    center_temp = temp.metadata.General.get_item("notes")
    print(center_temp)
    temp = temp.data
    print(temp.shape)
       
    resize_temp = []
    len_profile = []
    for j in range(temp.shape[0]):
        for k in range(temp.shape[1]):
            radial_temp = radial_average(temp[j, k], profile_resolution=1.0, center=center_temp)
            resize_temp.append(radial_temp)
            len_profile.append(len(radial_temp))
    
    min_sdim = np.min(len_profile)
    for i, arr in enumerate(resize_temp):
        resize_temp[i] = resize_temp[i][:min_sdim]
    resize_temp = np.asarray(resize_temp)
    resize_temp = resize_temp.reshape(temp.shape[0], temp.shape[1], -1)
    resize_temp /= np.max(resize_temp)
    print(resize_temp.shape)
    print(np.max(resize_temp))
    print(np.min(resize_temp))
    print(np.mean(resize_temp))
    print(np.median(resize_temp))
    
    data_storage.append(resize_temp)

del(temp)
del(resize_temp)

In [ ]:
data_shape = []
for arr in data_storage:
    data_shape.append(arr.shape)

In [ ]:
%matplotlib qt
check_center = hys.signals.Signal1D(data_storage[0][:, :, 20:])
check_center.plot()

In [ ]:
cut_point = 20
num_img = len(data_storage)

data_shape = np.asarray(data_shape)
print(data_shape)
s_dim = np.min(data_shape[:, 2])
dataset = []
data_shape_new = []

cluster_size_y = 1
cluster_size_x = 1
stride_y = 1
stride_x = 1

for img in data_storage:
    height = len(range(0, img.shape[0]-cluster_size_y+1, stride_y))
    width = len(range(0, img.shape[1]-cluster_size_x+1, stride_x))
    data_shape_new.append([height, width])
    temp= []
    for i in range(0, img.shape[0]-cluster_size_y+1, stride_y):
        for j in range(0, img.shape[1]-cluster_size_x+1, stride_x):
            #temp_sum = warp_scale(np.sum(img[i:i+cluster_size_y, j:j+cluster_size_x, cut_point:], axis=(0, 1)))
            temp_sum = np.sum(img[i:i+cluster_size_y, j:j+cluster_size_x, cut_point:s_dim], axis=(0, 1))
            temp.append(temp_sum)
    temp = np.asarray(temp).reshape(height, width, s_dim-cut_point)
    dataset.append(temp)

s_dim = s_dim - cut_point
data_shape_new = np.asarray(data_shape_new)
print(data_shape_new)
print(s_dim)

In [ ]:
%matplotlib inline
img_storage = []

for i in range(num_img):
    img_storage.append(np.sum(data_storage[i], axis=2))
            
for i in range(num_img):
    plt.imshow(img_storage[i])
    plt.show()

In [ ]:
def colortable(colors, title, sort_colors=True, emptycols=0):
    
    by_hsv = ((tuple(mcolors.rgb_to_hsv(mcolors.to_rgba(color)[:3])), name)
                    for name, color in colors.items())
    if sort_colors is True:
        by_hsv = sorted(by_hsv)
    names = [name for hsv, name in by_hsv]
    
    return names

color_name = colortable(mcolors.CSS4_COLORS, "CSS Colors")
num_color = len(color_name)
print(len(color_name))

ri = np.random.choice(num_color, num_color, replace=False)
ri = np.sort(ri)
print(ri)
color_cand_4 = {}
for i in range(num_color):
    color_cand_4[i] = color_name[ri[i]]

In [ ]:
# coloring by vertical lines
class_group = []
color_group = []
for i in range(num_img):
    for j in range(data_shape_new[i, 0]):
        for k in range(data_shape_new[i, 1]):
            color_group.append(color_cand_4[k])

In [ ]:
# coloring by horizontal lines
class_group = []
color_group = []
for i in range(num_img):
    for j in range(data_shape_new[i, 0]):
        for k in range(data_shape_new[i, 1]):
            color_group.append(color_cand_4[i])

In [ ]:
# only one color
class_group = []
color_group = []
for i in range(num_img):
    for j in range(data_shape_new[i, 0]):
        for k in range(data_shape_new[i, 1]):
            color_group.append("red")

In [ ]:
dataset_input = []
for i in range(num_img):
    dataset_input.extend(dataset[i].reshape(-1, s_dim).tolist())
    
dataset_input = np.asarray(dataset_input)
print(dataset_input.shape)

In [ ]:
def nmf(data_set, n_comp=5, vis_comp=False, projection=False, inverse=False, initialize="nndsvda"):
    print("Doing NMF...")

    nmf = NMF(n_components=n_comp, init=initialize, solver="mu", max_iter=1000, random_state=56, verbose=True)

    reduced_projection = nmf.fit_transform(data_set)
    restore = nmf.inverse_transform(reduced_projection)
    
    if vis_comp:
        n1 = n_comp
        fig1, ax1 = plt.subplots(n1, 1, figsize=(10, 10))
        temp_flat = ax1.flat
        for i, ax in enumerate(temp_flat[:n1]):
            ax.plot(nmf.components_[i], "k-", c="black")
            ax.grid()

    if projection:
        fig2= plt.figure(figsize=(10,10))
        ax2 = fig2.add_subplot(221)
        ax2.scatter(reduced_projection[:, 0], reduced_projection[:, 1], s=1.5, c="black", alpha=0.5)
        ax2.grid()
        ax2.set_xlabel("component 1")
        ax2.set_ylabel("component 2")

        ax3 = fig2.add_subplot(222)
        ax3.scatter(reduced_projection[:, 1], reduced_projection[:, 2], s=1.5, c="black", alpha=0.5)
        ax3.grid()
        ax3.set_xlabel("component 2")
        ax3.set_ylabel("component 3")

        ax4 = fig2.add_subplot(223)
        ax4.scatter(reduced_projection[:, 0], reduced_projection[:, 2], s=1.5, c="black", alpha=0.5)
        ax4.grid()
        ax4.set_xlabel("component 1")
        ax4.set_ylabel("component 3")
        
        if n_comp > 2:
            ax5 = fig2.add_subplot(224, projection="3d")
            ax5.scatter(reduced_projection[:, 0], reduced_projection[:, 1],reduced_projection[:, 2], zdir="z", s=1.5, c="black", alpha=0.5)
            ax5.grid()
            ax5.set_xlabel("component 1")
            ax5.set_ylabel("component 2")
            ax5.set_zlabel("component 3")

    return nmf.components_, reduced_projection, restore

In [ ]:
%matplotlib inline
num_comp = 4
comp_vectors, projections, restoration = nmf(dataset_input, n_comp=num_comp, 
                                             vis_comp=False, projection=False, initialize="nndsvda")
plt.show()

In [ ]:
%matplotlib inline
print(comp_vectors.shape)
fig, ax = plt.subplots(num_comp, 1, figsize=(5, 8))
for i, a in enumerate(ax):
    a.plot(comp_vectors[i], "k-", c="black")
    a.grid()
    #a.set_title("%d component"%(i+1), fontsize=15)
    #a.set_xlabel("eV", fontsize=15)
fig.tight_layout()
plt.show()

In [ ]:
plt.close(fig)

In [ ]:
projections_copy = projections.copy()
print(projections_copy.shape)
projection_reshape = []
for i in range(num_img):
    temp = projections_copy[:int(data_shape_new[i, 0]*data_shape_new[i, 1]), :]
    print(temp.shape)
    projections_copy = np.delete(projections_copy, range(int(data_shape_new[i, 0]*data_shape_new[i, 1])), axis=0)
    print(projections_copy.shape)
    temp = np.reshape(temp, (data_shape_new[i, 0], data_shape_new[i, 1], -1))
    print(temp.shape)
    projection_reshape.append(temp)

In [ ]:
%matplotlib qt
# for many images
fig1, ax1 = plt.subplots(1, num_img, figsize=(8, 6))
    
def map_selector(comp):
    for i, axs in enumerate(ax1):
        axs.cla()
        axs.imshow(projection_reshape[i][:, :, comp], cmap="gray")
        #axs.set_title("component %d map"%(comp+1), fontsize=10)
        axs.axis("off")
    fig1.canvas.draw()
    fig1.tight_layout()
    
interact(map_selector, comp=widgets.IntSlider(min=0, max=num_comp-1, step=1, value=0))

In [ ]:
plt.close(fig1)

In [ ]:
%matplotlib qt
fig2, ax2 = plt.subplots(1, 1, figsize=(10, 10))

def projection(c1, c2):
    ax2.cla()
    ax2.scatter(projections[:, c1], projections[:, c2], s=5, c=color_group, alpha=0.5)
    ax2.grid()
    ax2.set_xlabel("component %d"%(c1+1), fontsize=20)
    ax2.set_ylabel("component %d"%(c2+1), fontsize=20)
    fig2.canvas.draw()
    fig2.tight_layout()

x_widget = widgets.IntSlider(min=0, max=num_comp-1, step=1, value=0)
y_widget = widgets.IntSlider(min=0, max=num_comp-1, step=1, value=1)

interact(projection, c1=x_widget, c2=y_widget)
fig2.show()

In [ ]:
plt.close(fig2)

In [ ]:
%matplotlib qt
# for many images
from matplotlib.widgets import EllipseSelector

comp_v1 = 1
comp_v2 = 3

fig3, ax3 = plt.subplots(1, 3, figsize=(20, 8))
ax3[0].scatter(projections[:, comp_v1], projections[:, comp_v2], s=3.0, alpha=0.5, c=color_group)
ax3[0].tick_params(axis="both", labelsize=15)
ax3[0].grid()
ax3[1].scatter(projections[:, comp_v1], projections[:, comp_v2], s=3.0, alpha=0.5, cmap="Set1")
ax3[1].tick_params(axis="both", labelsize=15)
ax3[1].grid()
ax3[2].plot(np.zeros(s_dim))
ax3[2].tick_params(axis="both", labelsize=15)
ax3[2].grid()

fig4, ax4 = plt.subplots(2, num_img, figsize=(10, 5))

cluster = []

for i in range(num_img):
    ax4[0][i].imshow(img_storage[i], cmap="gray")
    ax4[0][i].axis("off")
    ax4[1][i].imshow(img_storage[i], cmap="gray")
    ax4[1][i].axis("off")

def ellipse_equation(cx, cy, a, b, x, y):
    return ((x-cx)/a)**2 + ((y-cy)/b)**2

def onselect(eclick, erelease):
    center_x = (erelease.xdata + eclick.xdata) / 2.0
    center_y = (erelease.ydata + eclick.ydata) / 2.0
    a = abs((erelease.xdata - eclick.xdata) / 2.0)
    b = abs((erelease.ydata - eclick.ydata) / 2.0)
    #print(center_x)
    #print(center_y)
    #print(a)
    #print(b)
    
    temp = []
    spectra_sum = np.zeros(s_dim)
    
    color = []
    for i in range(len(dataset_input)):
        if ellipse_equation(center_x, center_y, a, b, projections[i, comp_v1], projections[i, comp_v2]) <= 1:
            #print("wow")
            temp.append(1)
            spectra_sum += dataset_input[i]
            color.append(1)
        else:
            temp.append(0)
            color.append(0)
            
    ax3[1].cla()
    ax3[1].scatter(projections[:, comp_v1], projections[:, comp_v2], s=3.0, alpha=0.5, c=color, cmap="Set1")
    ax3[1].grid()
    
    ax3[2].cla()
    ax3[2].plot(warp_scale(spectra_sum))
    ax3[2].grid()
    
    temp = np.asarray(temp)
    #print(temp.shape)
    selected = []
    for i in range(num_img):
        selected_temp = temp[:int(data_shape_new[i, 0]*data_shape_new[i, 1])]
        temp = np.delete(temp, range(int(data_shape_new[i, 0]*data_shape_new[i, 1])))
        selected_temp = np.reshape(selected_temp, data_shape_new[i])
        selected.append(selected_temp)
    
    for i in range(num_img):
        ax4[1][i].imshow(selected[i], cmap="Set1")
        ax4[1][i].axis("off")
        
    fig3.canvas.draw()
    fig4.canvas.draw()
    fig3.tight_layout()
    fig4.tight_layout()
    
    cluster.append(selected)
        
ep = EllipseSelector(ax3[0], onselect, drawtype="line")
plt.show()

In [ ]:
plt.close(fig3)
plt.close(fig4)
print(len(cluster))

In [ ]:
selected = []
for i in range(num_img):
    selected.extend(cluster[-1][i].flatten().tolist())

In [ ]:
for i, s in enumerate(selected):
    if s == 0:
        dataset_input[i] *= 0

In [ ]:
num_sel = 3
selected = []
for i in range(num_sel):
    selected.append(cluster[-(i+1)])
    
selected.reverse()

In [ ]:
# compare the reconstructed signals by clusters

comp_array = np.arange(num_comp)
#comp_array = np.array([1, 2])

reconstructed = np.dot(projections[:, comp_array], comp_vectors[comp_array])
print(reconstructed.shape)
reconstructed_copy = reconstructed.copy()
reconstructed_reshape = []

for i in range(num_img):
    temp = reconstructed_copy[:int(data_shape_new[i, 0]*data_shape_new[i, 1]), :]
    print(temp.shape)
    reconstructed_copy = np.delete(reconstructed_copy, range(int(data_shape_new[i, 0]*data_shape_new[i, 1])), axis=0)
    print(projections_copy.shape)
    temp = np.reshape(temp, (data_shape_new[i, 0], data_shape_new[i, 1], -1))
    print(temp.shape)
    reconstructed_reshape.append(temp)   

#for i in range(num_img):
#    for j in range(data_shape_new[i, 0]):
#        for k in range(data_shape_new[i, 1]):
#            reconstructed_reshape[i][j, k] = warp_scale(reconstructed_reshape[i][j, k])

In [ ]:
# compare the original input signals by clusters

reconstructed_reshape = []
dataset_input_copy = dataset_input.copy()
for i in range(num_img):
    temp = dataset_input_copy[:int(data_shape_new[i, 0]*data_shape_new[i, 1]), :]
    dataset_input_copy = np.delete(dataset_input_copy, range(int(data_shape_new[i, 0]*data_shape_new[i, 1])), axis=0)
    temp = np.reshape(temp, (data_shape_new[i, 0], data_shape_new[i, 1], -1))
    reconstructed_reshape.append(temp)
    
#for i in range(num_img):
#    for j in range(data_shape_new[i, 0]):
#        for k in range(data_shape_new[i, 1]):
#            reconstructed_reshape[i][j, k] = warp_scale(reconstructed_reshape[i][j, k])

In [ ]:
# cluster vs. cluster in all individual images
%matplotlib inline
lines = np.zeros((num_img, num_sel, s_dim))
sel_num = np.zeros((num_img, num_sel))

for h in range(num_img):
    for i in range(num_sel):
        for j in range(data_shape_new[h, 0]):
            for k in range(data_shape_new[h, 1]):
                if selected[i][h][j, k] == 1:
                    sel_num[h, i] += 1
                    lines[h, i] += reconstructed_reshape[h][j, k]

print(sel_num) 
#for h in range(num_img):            
#    for i in range(num_sel):
#        lines[h, i] = warp_scale(lines[h, i])

for h in range(num_img):
    for i in range(num_sel):
        lines[h, i] /= sel_num[h, i]
        
lines /= np.max(lines)
            
fig5, ax5 = plt.subplots(1, 1, figsize=(10, 10))

for j in range(num_img):
    for i in range(num_sel):
        ax5.plot((lines[j, i]+j*0.20), label="image %d - line %d"%(j+1, i+1), c=color_cand_4[(i+1)*25])
        
ax5.grid()
ax5.set_facecolor("black")
fig5.legend()
fig5.tight_layout()
plt.show()

In [ ]:
plt.close(fig5)

In [ ]:
# cluster vs. cluster
%matplotlib inline
lines = np.zeros((num_sel, s_dim))
sel_num = np.zeros(num_sel)

for h in range(num_img):
    for i in range(num_sel):
        for j in range(data_shape_new[h, 0]):
            for k in range(data_shape_new[h, 1]):
                if selected[i][h][j, k] == 1:
                    sel_num[i] += 1
                    lines[i] += reconstructed_reshape[h][j, k]
print(sel_num)
lines_nor = np.zeros((num_sel, s_dim))
for i in range(num_sel):
    lines_nor[i] = warp_scale(lines[i])

for i in range(num_sel):
    lines[i] /= sel_num[i]
            
fig11, ax11 = plt.subplots(2, 1, figsize=(10, 20))

for i in range(num_sel):
    ax11[0].plot((lines[i]), label="line %d"%(i+1), c=color_cand_4[(i+1)*25])
    ax11[1].plot((lines_nor[i]), label="line %d"%(i+1), c=color_cand_4[(i+1)*25])
ax11[0].grid()
ax11[0].set_facecolor("black")
ax11[0].legend()
ax11[1].grid()
ax11[1].set_facecolor("black")
ax11[1].legend()
fig11.tight_layout()

plt.show()

In [ ]:
plt.close(fig11)

In [ ]:
# image vs. image depending on the component
%matplotlib qt
lines = np.zeros((num_img, num_sel, s_dim))
sel_num = np.zeros((num_img, num_sel))

for h in range(num_img):
    for i in range(num_sel):
        for j in range(data_shape_new[h, 0]):
            for k in range(data_shape_new[h, 1]):
                if selected[i][h][j, k] == 1:
                    sel_num[h, i] += 1
                    lines[h, i] += reconstructed_reshape[h][j, k]

print(sel_num) 
#for h in range(num_img):            
#    for i in range(num_sel):
#        lines[h, i] = warp_scale(lines[h, i])

for h in range(num_img):
    for i in range(num_sel):
        lines[h, i] /= sel_num[h, i]
        
#for h in range(num_img):
#    lines[h] /= np.max(lines[h])
            
fig6, ax6 = plt.subplots(1, 1, figsize=(10, 10))

def comp_selector(n_sel):
    ax6.cla()
    color = []
    for i in range(num_img):
        ax6.plot((lines[i, n_sel]), label="image %d"%(i+1), c=color_cand_4[(i+1)*17])
    ax6.grid()
    ax6.set_facecolor("black")
    fig6.canvas.draw()
    fig6.legend()
    fig6.tight_layout()
    
interact(comp_selector, n_sel=widgets.IntSlider(min=0, max=num_sel-1, step=1, value=0))
plt.show()

In [ ]:
plt.close(fig6)

In [ ]:
# horizontal lines
%matplotlib qt
line_evolve = []
for i in range(num_img):
    line_evolve.append(np.sum(reconstructed_reshape[i], axis=1))
    
line_evolve = np.asarray(line_evolve)
#print(line_evolve[0].shape)

for i in range(num_img):
    for j in range(data_shape_new[i, 1]):
        line_evolve[i][j] = warp_scale(line_evolve[i][j])
        
fig7, ax7 = plt.subplots(1, 1, figsize=(8, 12))
ax7.tick_params(axis="both", labelsize=15)
fig8, ax8 = plt.subplots(1, 1, figsize=(8, 10))

def img_selector(img_n, top, bottom):
    ax7.cla()
    ax8.cla()
    print("top: %d"%top)
    print("bottom: %d"%bottom)
    if bottom > data_shape_new[img_n, 0]:
        print("exceed the range of the image")
        
    elif top >= bottom:
        print("top must be smaller than bottom")
        
    else:
        for i in range(top, bottom, 1):
            ax7.plot((line_evolve[img_n][-i]+i*(5.0)), c=color_cand_4[i], label="line %d"%(i+1))
            ax8.plot(line_evolve[img_n][-i], c=color_cand_4[i], label="line %d"%(i+1))
    ax7.grid()
    ax7.set_facecolor("black")
    ax8.grid()
    ax8.set_facecolor("black")
    fig7.canvas.draw()
    fig8.canvas.draw()
    fig7.tight_layout()
    fig8.tight_layout()

print(data_shape_new)
interact(img_selector, img_n=widgets.IntSlider(min=0, max=num_img-1, step=1, value=0), 
        top=widgets.IntSlider(min=0, max=np.max(data_shape_new[:, 0]), step=1, value=0), 
        bottom=widgets.IntSlider(min=0, max=np.max(data_shape_new[:, 0]), step=1, value=10))

plt.show()

In [ ]:
plt.close(fig7)
plt.close(fig8)

In [ ]:
# vertical lines
%matplotlib qt
line_evolve = []
for i in range(num_img):
    line_evolve.append(np.sum(reconstructed_reshape[i], axis=0))
    
line_evolve = np.asarray(line_evolve)
#print(line_evolve[0].shape)

for i in range(num_img):
    for j in range(data_shape_new[i, 1]):
        line_evolve[i][j] = warp_scale(line_evolve[i][j])
        
fig9, ax9 = plt.subplots(1, 1, figsize=(8, 12))
fig10, ax10 = plt.subplots(1, 1, figsize=(8, 10))

def img_selector(img_n, left, right):
    ax9.cla()
    ax10.cla()
    print("left: %d"%left)
    print("right: %d"%right)
    if right > data_shape_new[img_n, 1]:
        print("exceed the range of the image")
        
    elif left >= right:
        print("left must be smaller than right")
        
    else:
        for i in range(left, right, 1):
            ax9.plot((line_evolve[img_n][-i]+i*1.0), c=color_cand_4[i], label="line %d"%(i+1))
            ax10.plot(line_evolve[img_n][-i], c=color_cand_4[i], label="line %d"%(i+1))
    ax9.grid()
    ax9.set_facecolor("black")
    ax10.grid()
    ax10.set_facecolor("black")
    fig9.canvas.draw()
    fig9.canvas.draw()
    fig10.tight_layout()
    fig10.tight_layout()

print(data_shape_new)
interact(img_selector, img_n=widgets.IntSlider(min=0, max=num_img-1, step=1, value=0), 
        left=widgets.IntSlider(min=0, max=np.max(data_shape_new[:, 1]), step=1, value=0), 
        right=widgets.IntSlider(min=0, max=np.max(data_shape_new[:, 1]), step=1, value=10))

plt.show()

In [ ]:
plt.close(fig9)
plt.close(fig10)

In [ ]:
for i in range(num_img):
    temp = hys.signals.Signal2D(reconstructed[i])
    temp_dm = hys.signals.Signal2D(np.rollaxis(np.rollaxis(reconstructed[i], 0, 3), 0, 3))
    temp.save(dm_files[i][:-5]+"_nmf_reconstructed.hdf5")
    temp_dm.save(dm_files[i][:-5]+"_nmf_reconstructed_dm_import.hdf5")

In [ ]:
print(projections[0, 0])
print(projections[0, 1])
X = np.stack((projections[:, 1], projections[:, 2]), axis=1)
print(X.shape)
print(X[0])

In [ ]:
# clustering using k-means
from sklearn.cluster import KMeans

k_cluster = KMeans(n_clusters=5, n_init=100, random_state=34).fit_predict(X)

fig = plt.figure(figsize=(10, 10))
plt.scatter(X[:, 0], X[:, 1], c=k_cluster, cmap="Accent")
plt.show()

In [ ]:
print(k_cluster.shape)
img_restored = np.reshape(k_cluster, (2, 69, 69))
print(img_restored[0].shape)

plt.imshow(img_restored[0], cmap="Accent")
plt.show()

In [ ]:
%matplotlib nbagg
# clustering using DBSCAN (Density-based Spatial Clustering of Applications with Noise)
from sklearn.cluster import DBSCAN
from sklearn import metrics

db = DBSCAN(eps=0.01, min_samples=50).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print('Estimated number of clusters: %d' % n_clusters_)

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [ ]:
print(type(labels))
print(labels.shape)
#print(labels[:500])
img_restored = np.reshape(labels, (2, 69, 69))

In [ ]:
plt.imshow(img_restored[0], cmap="Accent")
plt.show()

In [ ]:
%matplotlib nbagg
from matplotlib.widgets import EllipseSelector

widget = plt.figure(figsize=(6, 10))
ax1 = widget.add_subplot(211)
original_plot = ax1.scatter(projections[:, 1], projections[:, 2], s=1.0, alpha=0.5, cmap="Accent")

ax2 = widget.add_subplot(212)
ax2.imshow(img_storage[0], cmap="Accent")

def ellipse_equation(cx, cy, a, b, x, y):
    return ((x-cx)/a)**2 + ((y-cy)/b)**2

def onselect(eclick, erelease):
    center_x = (erelease.xdata + eclick.xdata) / 2.0
    center_y = (erelease.ydata + eclick.ydata) / 2.0
    a = abs((erelease.xdata - eclick.xdata) / 2.0)
    b = abs((erelease.ydata - eclick.ydata) / 2.0)
    #print(center_x)
    #print(center_y)
    #print(a)
    #print(b)
    
    ax1.cla()
    temp = np.zeros(img_storage[0].size)
    color = []
    for i in range(len(projections[:, 1])):
        if ellipse_equation(center_x, center_y, a, b, projections[i, 1], projections[i, 2]) <= 1:
            #print("wow")
            temp[i] = 1
            color.append(1)
        else:
            temp[i] = 0
            color.append(0)
    
    ax1.scatter(projections[:, 1], projections[:, 2], s=1.0, alpha=0.5, c=color, cmap="Accent")
    selected = np.reshape(temp, (img_shape[0], img_shape[1]))
    print(np.max(selected))
    ax2.imshow(selected, cmap="gray")
    widget.canvas.draw()
        
ep = EllipseSelector(ax1, onselect, drawtype="line")
plt.show()
#ax3 = widget.add_subplot(313)

In [ ]:
# when dimensions of data are different
data_storage_temp = []
for adr in dm_files:
    temp = hys.load(adr)
    print(temp.data.shape)
    data_storage_temp.append(temp.data)
    
num_img = len(data_storage_temp)

shapes = []
for arr in data_storage_temp:
    shapes.append(list(arr.shape))
shapes = np.asarray(shapes)

img_shape = [np.min(shapes[:, 0]), np.min(shapes[:, 1]), np.min(shapes[:, 2])]

data_storage = []
for arr in data_storage_temp:
    data_storage.append(arr[:img_shape[0], :img_shape[1], :img_shape[2]])